# **This Notebook is for IBM Data Science Capstone Project (Coursera)**

In [1]:
import pandas as pd
import numpy as np

print(' Hello Capstone Project Course!')

 Hello Capstone Project Course!


# **Segmenting and Clustering Neighborhoods in Toronto**
In this notebook we will be scrapping wikipedia to get the List of Postal Codes of Canada to build the Table (Pandas DataFrame) and later use it for clustering purposes

Importing required libraries

In [1]:
import pandas as pd
import numpy as np
import csv #For saving table as a csv file

Geting the data from the wikipedia in the form of Tables

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

That return 3 tables but we only want 1st one, So let's save it into a DataFrame named can

In [3]:
can = tables[0]
can.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Let's process this table to make it in proper table

In [4]:
headers = can.iloc[0]
headers.astype('str',inplace = True)
can.columns = headers
can.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [5]:
can.drop([0,1,2],axis=0, inplace = True)
can.reset_index(drop=True,inplace = True)
can.replace('Not assigned',np.nan,inplace = True)
can.dropna(thresh=2,inplace = True)
can.reset_index(drop=True)
can.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Now that we have a proper DataFrame Let's Process it as per our Requirements 

In [6]:
post = can['Postcode'].unique()

In [7]:
v = []

csv_file = open('canada.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode','Borough','Neighbourhood'])

for i in range(post.shape[0]):
    
    z = can[can.Postcode == post[i]]
    
    for j in range(z.shape[0]):
        
        v.append(z.iloc[0,0])
        v.append(z.iloc[j,1])
        v.append(z.iloc[j,2])
        
        if j == z.shape[0] - 1:
            
            vs = set(v)
            vs.remove(z.iloc[0,1])
            
            for k in range(post.shape[0]):
                
                if post[k] in vs:
                    vs.remove(post[k])
            
            postcode = z.iloc[0,0]
            borough = z.iloc[0,1]
            neighbourhood = list(vs)
            
            del(vs)
            v.clear()
            
            csv_writer.writerow([postcode,borough,', '.join(str(s) for s in neighbourhood)])
            
csv_file.close() 

Now we have a csv file in our Required format so let's import it as a DataFrame

In [8]:
canada = pd.read_csv('canada.csv')
canada.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,NaN


Since there is no null value in Postcode and Borough Let's check how many missing values(NaN) are present in Neighbourhood

In [9]:
canada.Neighbourhood.isnull().value_counts()

False    102
True       1
Name: Neighbourhood, dtype: int64

As their is only one missing value we can Directly replace it with Queen's Park

In [10]:
canada.fillna("Queen's Park", inplace=True)

In [11]:
canada.Neighbourhood.isnull().value_counts()

False    103
Name: Neighbourhood, dtype: int64

In [12]:
canada.shape

(103, 3)

Let's import Dataset containing values of latitute and longitude corresponding to Postal Codes 

In [13]:
lat_long = pd.read_csv('C:/PythonWork/Project/Geospatial_Coordinates.csv')

To merge the two DataFrames together column name of Primary Key must be same therefore let's rename both DataFrames column to PostalCode

In [14]:
canada.rename(columns={'Postcode':'PostalCode'},inplace=True)
lat_long.rename(columns={'Postal Code':'PostalCode'},inplace=True)

Merging the two DataFrames together to form one DataFrame named Neighbourhood_Canada

In [15]:
Neighbourhood_Canada = pd.merge(canada,lat_long,how='left',on='PostalCode')

In [16]:
Neighbourhood_Canada.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
